In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
SAMPLING_FREQ = 200

In [7]:
# load the data from csv files
def load_from_fragments(name):
    path = name + '_fragmentized.csv'
    data = pd.read_csv(path)
    return data

def compute_fft(data, sampling_freq):
    # Compute the length of the signal
    N = len(data)
    
    # Compute the frequencies corresponding to the FFT
    freqs = np.fft.fftfreq(N, d=1/sampling_freq)
    
    # Compute the FFT of the data
    fft_vals = np.fft.fft(data)
    
    return freqs, fft_vals

# plot the time signal for a given channel
def plot_time_signal(name, data, channel, save=False):
    plt.figure()
    plt.plot(data[channel])
    plt.xlabel('frame')
    plt.ylabel('Amplitude')
    plt.title('Time signal for channel ' + channel)
    if save:
        plt.savefig('./plots/timesignal/' + name + '/ts_channel_' + channel + '.png')
    plt.close()

# plot the frequency distribution for a given channel by using the FFT
def plot_frequency_distribution(name, data, channel, save=False):
    plt.figure()
    freqs, fft_vals = compute_fft(data[channel], SAMPLING_FREQ)
    plt.plot(freqs, np.abs(fft_vals))
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')
    plt.title('Frequency distribution for channel ' + channel)
    if save:
        plt.savefig('./plots/frequency/' + name + '/fd_channel_' + channel + '.png')
    plt.close()

In [8]:
# load the data
data_029b0RMT = load_from_fragments('029b0RMT')
data_117h0RMT = load_from_fragments('117h0RMT')

print(data_029b0RMT.shape)
print(data_117h0RMT.shape)

(23478, 43)
(21209, 45)


In [9]:
# plot time signals and frequency distributions for all channels
# for key in data_029b0RMT.keys():
#     plot_time_signal('029b0RMT', data_029b0RMT, key, save=True)
#     plot_frequency_distribution('029b0RMT', data_029b0RMT, key, save=True)
# for key in data_117h0RMT.keys():
#     plot_time_signal('117h0RMT', data_117h0RMT, key, save=True)
#     plot_frequency_distribution('117h0RMT', data_117h0RMT, key, save=True)

In [10]:
def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    filtered_data = filtfilt(b, a, data)
    return filtered_data

In [11]:
# delta_filtered_029b0RMT = butter_bandpass_filter(data_029b0RMT, 0.5, 4, SAMPLING_FREQ)
# delta_filtered_117h0RMT = butter_bandpass_filter(data_117h0RMT, 0.5, 4, SAMPLING_FREQ)
delta_filtered_029b0RMT = pd.DataFrame()
delta_filtered_117h0RMT = pd.DataFrame()

for channel in data_029b0RMT.columns[:-1]:
    delta_filtered_029b0RMT[channel] = butter_bandpass_filter(data_029b0RMT[channel], 0.5, 4, SAMPLING_FREQ)

for channel in data_117h0RMT.columns[:-1]:
    delta_filtered_117h0RMT[channel] = butter_bandpass_filter(data_117h0RMT[channel], 0.5, 4, SAMPLING_FREQ)

delta_filtered_029b0RMT['fragment'] = data_029b0RMT['fragment']
delta_filtered_117h0RMT['fragment'] = data_117h0RMT['fragment']

print(delta_filtered_029b0RMT.shape)
print(delta_filtered_117h0RMT.shape)

# save the filtered data
delta_filtered_029b0RMT.to_csv('./data/delta_filtered_029b0RMT.csv', index=False)
delta_filtered_117h0RMT.to_csv('./data/delta_filtered_117h0RMT.csv', index=False)

(23478, 43)
(21209, 45)
